In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [2]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

In [3]:
# One must patch the DPO Trainer first!
from unsloth import PatchDPOTrainer
PatchDPOTrainer()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


## DPO Zephyr Unsloth

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/zephyr-sft-bnb-4bit", # Choose ANY! eg mistralai/Mistral-7B-Instruct-v0.2
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.9.post4: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/511 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [5]:
import os
import re
from typing import List, Literal, Optional

from datasets import DatasetDict, concatenate_datasets, load_dataset, load_from_disk
from datasets.builder import DatasetGenerationError


DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"


def apply_chat_template(
    example, tokenizer, task: Literal["sft", "generation", "rm", "dpo"] = "sft", assistant_prefix="<|assistant|>\n"
):
    def _strip_prefix(s, pattern):
        # Use re.escape to escape any special characters in the pattern
        return re.sub(f"^{re.escape(pattern)}", "", s)

    if task in ["sft", "generation"]:
        messages = example["messages"]
        # We add an empty system message if there is none
        if messages[0]["role"] != "system":
            messages.insert(0, {"role": "system", "content": ""})
        example["text"] = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True if task == "generation" else False
        )
    elif task == "rm":
        if all(k in example.keys() for k in ("chosen", "rejected")):
            chosen_messages = example["chosen"]
            rejected_messages = example["rejected"]
            # We add an empty system message if there is none
            if chosen_messages[0]["role"] != "system":
                chosen_messages.insert(0, {"role": "system", "content": ""})
            if rejected_messages[0]["role"] != "system":
                rejected_messages.insert(0, {"role": "system", "content": ""})
            example["text_chosen"] = tokenizer.apply_chat_template(chosen_messages, tokenize=False)
            example["text_rejected"] = tokenizer.apply_chat_template(rejected_messages, tokenize=False)
        else:
            raise ValueError(
                f"Could not format example as dialogue for `rm` task! Require `[chosen, rejected]` keys but found {list(example.keys())}"
            )
    elif task == "dpo":
        if all(k in example.keys() for k in ("chosen", "rejected")):
            # Compared to reward modeling, we filter out the prompt, so the text is everything after the last assistant token
            prompt_messages = [[msg for msg in example["chosen"] if msg["role"] == "user"][0]]
            # Insert system message
            if example["chosen"][0]["role"] != "system":
                prompt_messages.insert(0, {"role": "system", "content": ""})
            else:
                prompt_messages.insert(0, example["chosen"][0])
            # TODO: handle case where chosen/rejected also have system messages
            chosen_messages = example["chosen"][1:]
            rejected_messages = example["rejected"][1:]
            example["text_chosen"] = tokenizer.apply_chat_template(chosen_messages, tokenize=False)
            example["text_rejected"] = tokenizer.apply_chat_template(rejected_messages, tokenize=False)
            example["text_prompt"] = tokenizer.apply_chat_template(
                prompt_messages, tokenize=False, add_generation_prompt=True
            )
            example["text_chosen"] = _strip_prefix(example["text_chosen"], assistant_prefix)
            example["text_rejected"] = _strip_prefix(example["text_rejected"], assistant_prefix)
        else:
            raise ValueError(
                f"Could not format example as dialogue for `dpo` task! Require `[chosen, rejected]` keys but found {list(example.keys())}"
            )
    else:
        raise ValueError(
            f"Task {task} not supported, please ensure that the provided task is one of {['sft', 'generation', 'rm', 'dpo']}"
        )
    return example


def get_datasets(
    data_config: dict,
    splits: List[str] = ["train", "test"],
    shuffle: bool = True,
) -> DatasetDict:
    """
    Loads one or more datasets with varying training set proportions.

    Args:
        data_config (`DataArguments` or `dict`):
            Dataset configuration and split proportions.
        splits (`List[str]`, *optional*, defaults to `['train', 'test']`):
            Dataset splits to load and mix. Assumes the splits exist in all datasets and have a `train_` or `test_` prefix.
        shuffle (`bool`, *optional*, defaults to `True`):
            Whether to shuffle the training and testing/validation data.

    Returns
        [`DatasetDict`]: The dataset dictionary containing the loaded datasets.
    """

    if type(data_config) is dict:
        # Structure of the input is:
        #     dataset_mixer = {
        #             "dataset1": 0.5,
        #             "dataset1": 0.3,
        #             "dataset1": 0.2,
        #         }
        dataset_mixer = data_config
    else:
        raise ValueError(f"Data config {data_config} not recognized.")

    raw_datasets = mix_datasets(dataset_mixer, splits=splits, shuffle=shuffle)
    return raw_datasets


def mix_datasets(dataset_mixer: dict, splits: Optional[List[str]] = None, shuffle=True) -> DatasetDict:
    """
    Loads and mixes datasets according to proportions specified in `dataset_mixer`.

    Args:
        dataset_mixer (`dict`):
            Dictionary containing the dataset names and their training proportions. By default, all test proportions are 1.
        splits (Optional[List[str]], *optional*, defaults to `None`):
            Dataset splits to load and mix. Assumes the splits exist in all datasets and have a `train_` or `test_` prefix.
        shuffle (`bool`, *optional*, defaults to `True`):
            Whether to shuffle the training and testing/validation data.
    """
    raw_datasets = DatasetDict()
    raw_train_datasets = []
    raw_val_datasets = []
    fracs = []
    for ds, frac in dataset_mixer.items():
        fracs.append(frac)
        for split in splits:
            try:
                # Try first if dataset on a Hub repo
                dataset = load_dataset(ds, split=split)
            except DatasetGenerationError:
                # If not, check local dataset
                dataset = load_from_disk(os.path.join(ds, split))

            if "train" in split:
                raw_train_datasets.append(dataset)
            elif "test" in split:
                raw_val_datasets.append(dataset)
            else:
                raise ValueError(f"Split type {split} not recognized as one of test or train.")

    if any(frac < 0 for frac in fracs):
        raise ValueError("Dataset fractions cannot be negative.")

    if len(raw_train_datasets) > 0:
        train_subsets = []
        for dataset, frac in zip(raw_train_datasets, fracs):
            train_subset = dataset.select(range(int(frac * len(dataset))))
            train_subsets.append(train_subset)
        if shuffle:
            raw_datasets["train"] = concatenate_datasets(train_subsets).shuffle(seed=42)
        else:
            raw_datasets["train"] = concatenate_datasets(train_subsets)
    # No subsampling for test datasets to enable fair comparison across models
    if len(raw_val_datasets) > 0:
        if shuffle:
            raw_datasets["test"] = concatenate_datasets(raw_val_datasets).shuffle(seed=42)
        else:
            raw_datasets["test"] = concatenate_datasets(raw_val_datasets)

    if len(raw_datasets) == 0:
        raise ValueError(
            f"Dataset {dataset_mixer} not recognized with split {split}. Check the dataset has been correctly formatted."
        )

    return raw_datasets

In [6]:
raw_datasets = get_datasets(
    {"HuggingFaceH4/ultrafeedback_binarized" : 0.005}, # 0.5% sampled
    splits = ["train_prefs", "test_prefs"],
)
column_names = list(raw_datasets["train"].features)

raw_datasets = raw_datasets.map(
    apply_chat_template,
    fn_kwargs = {"tokenizer": tokenizer, "task": "dpo"},
    num_proc = 12,
    remove_columns = column_names,
    desc = "Formatting comparisons with prompt template",
)

# Replace column names with what TRL needs, text_chosen -> chosen and text_rejected -> rejected
for split in ["train", "test"]:
    raw_datasets[split] = raw_datasets[split].rename_columns(
        {"text_prompt": "prompt", "text_chosen": "chosen", "text_rejected": "rejected"}
    )

README.md:   0%|          | 0.00/6.76k [00:00<?, ?B/s]

train_prefs-00000-of-00001.parquet:   0%|          | 0.00/226M [00:00<?, ?B/s]

test_prefs-00000-of-00001.parquet:   0%|          | 0.00/7.29M [00:00<?, ?B/s]

test_sft-00000-of-00001.parquet:   0%|          | 0.00/3.72M [00:00<?, ?B/s]

train_gen-00000-of-00001.parquet:   0%|          | 0.00/184M [00:00<?, ?B/s]

...

Generating test_sft split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Formatting comparisons with prompt template (num_proc=12):   0%|          | 0/305 [00:00<?, ? examples/s]

Formatting comparisons with prompt template (num_proc=12):   0%|          | 0/2000 [00:00<?, ? examples/s]

In [7]:
import pprint
row = raw_datasets["train"][5]
pprint.pprint(row["rejected"])

'Sure, I would love to play.</s>\n'


## Add LoRA Adapters


In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 64,
    lora_dropout = 0, # Currently only supports dropout = 0
    bias = "none",    # Currently only supports bias = "none"
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Train the DPO model

In [9]:
# One must patch the DPO Trainer first!
from unsloth import PatchDPOTrainer
PatchDPOTrainer()

In [10]:
from transformers import TrainingArguments
from trl import DPOTrainer, DPOConfig
from unsloth import is_bfloat16_supported

dpo_trainer = DPOTrainer(
    model = model,
    ref_model = None,
    args = DPOConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 16,
        warmup_ratio = 0.02,
        num_train_epochs = 1,
        learning_rate = 5e-6,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.0,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
    ),
    beta = 0.1,
    train_dataset = raw_datasets["train"],
    # eval_dataset = raw_datasets["test"],
    tokenizer = tokenizer,
    max_length = 512,
    max_prompt_length = 256,
)

Tokenizing train dataset:   0%|          | 0/305 [00:00<?, ? examples/s]

In [11]:
dpo_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 305 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 16
\        /    Total batch size = 32 | Total steps = 9
 "-____-"     Number of trainable parameters = 167,772,160
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / rejected,logps / chosen,logits / rejected,logits / chosen


TrainOutput(global_step=9, training_loss=0.6621550983852811, metrics={'train_runtime': 836.7464, 'train_samples_per_second': 0.365, 'train_steps_per_second': 0.011, 'total_flos': 0.0, 'train_loss': 0.6621550983852811, 'epoch': 0.9411764705882353})

## ORPOR Unsloth Example

In [12]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

### Add Lora Adapters


In [13]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [14]:
# The data must be formatted with appropriate prompt template first.
# See details here: https://github.com/huggingface/trl/blob/main/examples/scripts/orpo.py

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def format_prompt(sample):
    instruction = sample["instruction"]
    input       = sample["input"]
    accepted    = sample["accepted"]
    rejected    = sample["rejected"]

    # ORPOTrainer expects prompt/chosen/rejected keys
    # See: https://huggingface.co/docs/trl/main/en/orpo_trainer
    sample["prompt"]   = alpaca_prompt.format(instruction, input, "")
    sample["chosen"]   = accepted + EOS_TOKEN
    sample["rejected"] = rejected + EOS_TOKEN
    return sample
pass

from datasets import load_dataset
dataset = load_dataset("reciperesearch/dolphin-sft-v0.1-preference")["train"]
dataset = dataset.map(format_prompt,)

README.md:   0%|          | 0.00/490 [00:00<?, ?B/s]

dpo_fixed.jsonl:   0%|          | 0.00/34.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

### Print Random Item from Dataset

In [15]:
import pprint
row = dataset[1]
print('ACCEPTED: ' + '=' * 50)
pprint.pprint(row["chosen"])


ACCEPTED: ==================================================
('Question: What makes Xkcd comics popular among internet users?\n'
 '\n'
 'Answer: Xkcd comics are popular among internet users because of their clever '
 'humor, relatable themes, and minimalist art style. They often cover topics '
 'like science, technology, and life experiences, making them appealing to a '
 'broad audience.<|end_of_text|>')


In [16]:
# Enable reward modelling stats
from unsloth import PatchDPOTrainer
PatchDPOTrainer()

### Train Model
Using Huggingface TRL's `ORPOTrainer`, we do 60 steps to speed things up. However, we can set num_train_epochs = 1 for a full run, and turn off max_steps=None. TRL's `DPOTrainer` is also supported!

In [17]:
from trl import ORPOConfig, ORPOTrainer
from unsloth import is_bfloat16_supported

orpo_trainer = ORPOTrainer(
    model = model,
    train_dataset = dataset,
    tokenizer = tokenizer,
    args = ORPOConfig(
        max_length = 512,
        max_prompt_length = max_seq_length//2,
        max_completion_length = max_seq_length//2,
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        beta = 0.1,
        logging_steps = 100,
        optim = "adamw_8bit",
        lr_scheduler_type = "linear",
        max_steps = 5, # Change to num_train_epochs = 1 for full training runs
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        output_dir = "outputs",
    ),
)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [18]:
#orpo_trainer.train()

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [19]:
def print_formatted_prompt(prompts):
    """
    Format and print the given prompt in a readable format.

    :param prompt: A single string prompt that includes instruction, input, and output.
    """
    # Remove special tags
    prompt = prompts[0]
    prompt_cleaned = prompt.replace("<|begin_of_text|>", "").replace("<|end_of_text|>", "")

    # Split into sections: Instruction, Input, Output
    sections = prompt_cleaned.split("### ")

    # Store formatted output components
    formatted_output = {}

    # Parse the sections
    for section in sections:
        if section.startswith("Instruction:"):
            formatted_output["Instruction"] = section.replace("Instruction:\n", "").strip()
        elif section.startswith("Input:"):
            formatted_output["Input"] = section.replace("Input:\n", "").strip()
        elif section.startswith("Output:"):
            formatted_output["Output"] = section.replace("Output:\n", "").strip()

    # Print each section in a clean format
    print("-----Formatted Prompt Output-----")
    print("Instruction:\n", formatted_output.get("Instruction", "N/A"))
    print("\nInput:\n", formatted_output.get("Input", "N/A"))
    print("\nOutput:\n", formatted_output.get("Output", "N/A"))
    print("---------------------------------\n")

In [20]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\n11\n\n### Explanation:\nThe next number in the fibonnaci sequence is 11.<|end_of_text|>']

#### Define Basic Template for Alpaca Prompt

In [21]:
# Define a basic template for alpaca_prompt
alpaca_prompt = "### Instruction:\n{}\n### Input:\n{}\n### Output:\n{}"
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

#### Sentiment Analysis

In [22]:
inputs_sentiment = tokenizer(
    [
        alpaca_prompt.format(
            "Analyze the sentiment of the given text. Return either Positive, Negative, or Neutral.",  # Instruction
            "I didn't like it!",  # Input text for sentiment analysis
            "",  # Output - leave this blank for generation!
        )
    ], return_tensors="pt"
).to("cuda")
outputs_sentiment = model.generate(**inputs_sentiment, max_new_tokens=32, use_cache=True)

In [23]:
print("Sentiment Analysis Result:")
print_formatted_prompt(tokenizer.batch_decode(outputs_sentiment))

Sentiment Analysis Result:
-----Formatted Prompt Output-----
Instruction:
 Analyze the sentiment of the given text. Return either Positive, Negative, or Neutral.

Input:
 I didn't like it!

Output:
 Negative
---------------------------------



#### (NER)

In [24]:
inputs_ner = tokenizer(
    [
        alpaca_prompt.format(
            "Identify named entities (i.e. Person, Organizations, and Locations) in the given text. The output should be in list format (i.e. [People: Charles, Organizations: NATO, Locations: Cuba])",  # Instruction
            "Ratan tata is one of the kindest person in the world and used to live in india",  # Input text for NER
            "",  # Output - leave this blank for generation!
        )
    ], return_tensors="pt"
).to("cuda")
outputs_ner = model.generate(**inputs_ner, max_new_tokens=64, use_cache=True)

In [25]:
print("Named Entity Recognition Result:")
print_formatted_prompt(tokenizer.batch_decode(outputs_ner))

Named Entity Recognition Result:
-----Formatted Prompt Output-----
Instruction:
 Identify named entities (i.e. Person, Organizations, and Locations) in the given text. The output should be in list format (i.e. [People: Charles, Organizations: NATO, Locations: Cuba])

Input:
 Ratan tata is one of the kindest person in the world and used to live in india

Output:
 [People: Ratan, Organizations: NATO, Locations: india]
---------------------------------



#### Text Classification

In [26]:
inputs_classification = tokenizer(
    [
        alpaca_prompt.format(
            "Classify the given text into one of the following categories: Politics, Sports, Technology, Health, or Entertainment. Your output should only be one of these categories, whichever most closely aligns with the input.",  # Instruction
            "I love badminton.",  # Input text for classification
            "",  # Output - leave this blank for generation!
        )
    ], return_tensors="pt"
).to("cuda")
outputs_classification = model.generate(**inputs_classification, max_new_tokens=32, use_cache=True)

In [27]:
print("Text Classification Result:")
print_formatted_prompt(tokenizer.batch_decode(outputs_classification))

Text Classification Result:
-----Formatted Prompt Output-----
Instruction:
 Classify the given text into one of the following categories: Politics, Sports, Technology, Health, or Entertainment. Your output should only be one of these categories, whichever most closely aligns with the input.

Input:
 I love badminton.

Output:
 Sports
---------------------------------



#### Text Summarization

In [40]:
inputs_summarization = tokenizer(
    [
        alpaca_prompt.format(
            "Summarize the following article into one sentence. It is important that only one sentence is used.",  # Instruction
            "In recent times, image segmentation has been involving everywhere including disease diagnosis to autonomous vehicle driving. In computer vision, this image segmentation is one of the vital works and it is relatively complicated than other vision undertakings as it needs low-level spatial data. Especially, Deep Learning has impacted the field of segmentation incredibly and gave us today different successful models. The deep learning associated Generated Adversarial Networks (GAN) has presenting remarkable outcomes on image segmentation. In this study, the authors have presented a systematic review analysis on recent publications of GAN models and their applications. Three libraries such as Embase (Scopus), WoS, and PubMed have been considered for searching the relevant papers available in this area. Search outcomes have identified 2084 documents, after two-phase screening 52 potential records are included for final review. The following applications of GAN have been emerged: 3D object generation, medicine, pandemics, image processing, face detection, texture transfer, and traffic controlling. Before 2016, research in this field was limited and thereafter its practical usage came into existence worldwide. The present study also envisions the challenges associated with GAN and paves the path for future research in this realm.",  # Input text for summarization
            "",  # Output - leave this blank for generation!
        )
    ], return_tensors="pt"
).to("cuda")
outputs_summarization = model.generate(**inputs_summarization, max_new_tokens=64, use_cache=True)

In [41]:
print("Text Summarization Result:")
print_formatted_prompt(tokenizer.batch_decode(outputs_summarization))

Text Summarization Result:
-----Formatted Prompt Output-----
Instruction:
 Summarize the following article into one sentence. It is important that only one sentence is used.

Input:
 In recent times, image segmentation has been involving everywhere including disease diagnosis to autonomous vehicle driving. In computer vision, this image segmentation is one of the vital works and it is relatively complicated than other vision undertakings as it needs low-level spatial data. Especially, Deep Learning has impacted the field of segmentation incredibly and gave us today different successful models. The deep learning associated Generated Adversarial Networks (GAN) has presenting remarkable outcomes on image segmentation. In this study, the authors have presented a systematic review analysis on recent publications of GAN models and their applications. Three libraries such as Embase (Scopus), WoS, and PubMed have been considered for searching the relevant papers available in this area. Search 

#### Question Answering

In [38]:
inputs_question_answering = tokenizer(
    [
        alpaca_prompt.format(
            "Answer the following question based on the context provided.",  # Instruction
            "Context: An immense mausoleum of white marble, built in Agra between 1631 and 1648 by order of the Mughal emperor Shah Jahan in memory of his favourite wife.\nQuestion: Where is the Taj Mahal located?",  # Input context and question
            "",  # Output - leave this blank for generation!
        )
    ], return_tensors="pt"
).to("cuda")
outputs_question_answering = model.generate(**inputs_question_answering, max_new_tokens=32, use_cache=True)


In [39]:
print("Question Answering Result:")
print_formatted_prompt(tokenizer.batch_decode(outputs_question_answering))

Question Answering Result:
-----Formatted Prompt Output-----
Instruction:
 Answer the following question based on the context provided.

Input:
 Context: An immense mausoleum of white marble, built in Agra between 1631 and 1648 by order of the Mughal emperor Shah Jahan in memory of his favourite wife.
Question: Where is the Taj Mahal located?

Output:
 Agra, India
---------------------------------



#### Information Extraction

In [50]:
inputs_information_extraction = tokenizer(
    [
        alpaca_prompt.format(
            "Extract key pieces of information such as dates, and names from the given text.",  # Instruction
            "Barack Hussein Obama II[a] (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. As a member of the Democratic Party, he was the first African-American president in U.S. history.",  # Input text for information extraction
            "",  # Output - leave this blank for generation!
        )
    ], return_tensors="pt"
).to("cuda")
outputs_information_extraction = model.generate(**inputs_information_extraction, max_new_tokens=64, use_cache=True)


In [51]:
print("Information Extraction Result:")
print_formatted_prompt(tokenizer.batch_decode(outputs_information_extraction))

Information Extraction Result:
-----Formatted Prompt Output-----
Instruction:
 Extract key pieces of information such as dates, and names from the given text.

Input:
 Barack Hussein Obama II[a] (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. As a member of the Democratic Party, he was the first African-American president in U.S. history.

Output:
 August 4, 1961, Barack Hussein Obama II, Democratic Party
---------------------------------



#### Topic Modeling

In [52]:
inputs_topic_modeling = tokenizer(
    [
        alpaca_prompt.format(
            "Identify the main topic of the given text.",  # Instruction
            "Barack Hussein Obama II[a] (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. As a member of the Democratic Party, he was the first African-American president in U.S. history.",
            "",  # Output - leave this blank for generation!
        )
    ], return_tensors="pt"
).to("cuda")
outputs_topic_modeling = model.generate(**inputs_topic_modeling, max_new_tokens=32, use_cache=True)


In [53]:
print("Topic Modeling Result:")
print_formatted_prompt(tokenizer.batch_decode(outputs_topic_modeling))

Topic Modeling Result:
-----Formatted Prompt Output-----
Instruction:
 Identify the main topic of the given text.

Input:
 Barack Hussein Obama II[a] (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. As a member of the Democratic Party, he was the first African-American president in U.S. history.

Output:
 Barack Hussein Obama II[a] (born August 4, 1961) is an American politician who served as the 44th president of the United
---------------------------------



#### Text Generation

In [54]:
inputs_text_generation = tokenizer(
    [
        alpaca_prompt.format(
            "Generate a short creative story based on the given prompt.",  # Instruction
            "Once upon a time in London, there was a family living with 2 kids. They had a farm on contryside...",  # Input text for text generation
            "",  # Output - leave this blank for generation!
        )
    ], return_tensors="pt"
).to("cuda")
outputs_text_generation = model.generate(**inputs_text_generation, max_new_tokens=128, use_cache=True)

In [55]:
print("Text Generation Result:")
print_formatted_prompt(tokenizer.batch_decode(outputs_text_generation))

Text Generation Result:
-----Formatted Prompt Output-----
Instruction:
 Generate a short creative story based on the given prompt.

Input:
 Once upon a time in London, there was a family living with 2 kids. They had a farm on contryside...

Output:
 Once upon a time in London, there was a family living with 2 kids. They had a farm on contryside. Their father was a good farmer and he had a good farm. He had many animals and plants. He had cows, chickens, pigs, horses, and many more. He also had a lot of crops. He had corn, wheat, barley, and many more. He had a lot of animals and crops. He had a lot of food to eat. He had a lot of meat and vegetables. He had a lot of milk and eggs. He had a lot of food to eat. He had a
---------------------------------

